In [1]:
import pandas as pd
from iowalicor.data.preprocessing import slugify, get_dataset_slice

In [2]:
dataset_file = "../data/raw/Iowa_Liquor_Sales.csv"

In [4]:
dataset = get_dataset_slice(dataset_file, random_state=33, shuffle=True)
dataset.County.fillna("no_county", inplace=True)
dataset.County = [slugify(count) for count in dataset.County]
dataset.City = [slugify(cit) for cit in dataset.City]

In [5]:
dataset

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S05261400003,04/30/2012,4073.0,"Uptown Liquor, Llc",306 HWY 69 SOUTH,forest_city,50436,306 HWY 69 SOUTH\nFOREST CITY 50436\n(43.26179...,95.0,winnebago,...,11786.0,Black Velvet Traveler,12.0,750.0,$4.83,$7.44,12.0,$89.28,9.00,2.38
1,S22129500010,11/03/2014,4315.0,Fareway Stores #554 / Washington,301 NORTH MARION AVE,washington,52353,301 NORTH MARION AVE\nWASHINGTON 52353\n(41.30...,92.0,washington,...,42716.0,Malibu Coconut Rum,12.0,750.0,$7.49,$11.24,12.0,$134.88,9.00,2.38
2,S24431000005,03/10/2015,4010.0,Newman's Thriftway,900 S LOCUST ST,glenwood,51534,"900 S LOCUST ST\nGLENWOOD 51534\n(41.038205, -...",65.0,mills,...,57158.0,Chi-Chi's Strawberry Margarita W/tequila,6.0,1750.0,$6.50,$9.75,6.0,$58.50,10.50,2.77
3,S18352800003,04/22/2014,3691.0,Target Store T-1791 / Urbandale,11148 PLUM DR,urbandale,50322,"11148 PLUM DR\nURBANDALE 50322\n(41.646972, -9...",77.0,polk,...,52316.0,Christian Bros Brandy,12.0,750.0,$5.42,$8.38,12.0,$100.56,9.00,2.38
4,S11968900053,05/01/2013,2644.0,Hy-Vee Fort Dodge Wine and Spirits,1511 2ND AVE NORTH,fort_dodge,50501,1511 2ND AVE NORTH\nFORT DODGE 50501\n(42.5083...,94.0,webster,...,11786.0,Black Velvet Traveler,12.0,750.0,$5.23,$7.84,12.0,$94.08,9.00,2.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121811,S04010800022,02/09/2012,2567.0,Hy-Vee Drugstore / Davenport,2200 WEST KIMBERLY,davenport,52806,2200 WEST KIMBERLY\nDAVENPORT 52806\n(41.56015...,82.0,scott,...,36978.0,Nikolai Vodka,6.0,1750.0,$6.87,$10.30,6.0,$61.80,10.50,2.77
121812,S17973100024,03/19/2014,2200.0,Sac Liquor Store,619 E MAIN ST,sac_city,50583,"619 E MAIN ST\nSAC CITY 50583\n(42.421341, -94...",81.0,sac,...,89215.0,Jose Cuervo Tradicional Reposado Tequila,12.0,750.0,$14.90,$22.35,3.0,$67.05,2.25,0.59
121813,S15523100027,11/05/2013,2964.0,Dahl's / Ankeny,1802 N ANKENY BLVD,ankeny,50023,"1802 N ANKENY BLVD\nANKENY 50023\n(41.747345, ...",77.0,polk,...,5106.0,Grant's Blended Scotch,12.0,750.0,$10.50,$15.75,6.0,$94.50,4.50,1.19
121814,S10551100008,02/12/2013,2603.0,Hy-Vee Wine and Spirits / Bettendorf,2890 DEVILS GLEN ROAD,bettendorf,52722,2890 DEVILS GLEN ROAD\nBETTENDORF 52722\n(41.5...,82.0,scott,...,41693.0,Uv Blue (raspberry) Vodka,12.0,750.0,$6.50,$9.74,12.0,$116.88,9.00,2.38


In [9]:
dataset.City.count()

121816